<a href="https://colab.research.google.com/github/fruizpace/LarryGithub/blob/master/prueba_bio_fifa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import files
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.preprocessing import StandardScaler

In [2]:
def print_acc_rec_prec(y_true, y_pred):
  acc = accuracy_score(y_true, y_pred)
  rec = recall_score(y_true, y_pred)
  prec = precision_score(y_true, y_pred)
  print(f"Accuracy: {acc}")
  print(f"Recall: {rec}")
  print(f"Precision: {prec}")
  return

In [6]:
uploaded = files.upload()

Saving test.csv to test.csv


In [7]:
test = pd.read_csv("test.csv")

<ipython-input-7-11f06cf6c585>:1: DtypeWarning: Columns (24,107) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv("test.csv")


In [19]:
df = pd.read_csv("train.csv")

<ipython-input-19-ef2ff63de291>:1: DtypeWarning: Columns (103) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("train.csv")


In [23]:
df_train = df.query('starter.notnull()')

In [ ]:
df_train

In [25]:
# imputar NA en variables con la media de df_train y lo ponemos tb en df_test
df_train.value_eur = df_train.value_eur.fillna(df_train.value_eur.mean())
df_train.wage_eur = df_train.wage_eur.fillna(df_train.wage_eur.mean())

<ipython-input-25-310d3cd70329>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.value_eur = df_train.value_eur.fillna(df_train.value_eur.mean())
<ipython-input-25-310d3cd70329>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.wage_eur = df_train.wage_eur.fillna(df_train.wage_eur.mean())


In [28]:
df_train.wage_eur.isna().sum()

0

In [30]:
df_train.value_eur.isna().sum()

0

In [32]:
# imputar NA en variables con la media de df_train y lo ponemos tb en test
test.value_eur = test.value_eur.fillna(df_train.value_eur.mean())
test.wage_eur = test.wage_eur.fillna(df_train.wage_eur.mean())

In [34]:
test.value_eur.isna().sum()

0

In [44]:
# estandarizar las variables numericas que entraran en el modelo:
standard_scaler = StandardScaler()
standard_scaler.fit(df_train[["potential", "value_eur", "wage_eur", "age"]])
df_train[["potential", "value_eur", "wage_eur", "age"]] = standard_scaler.transform(df_train[["potential", "value_eur", "wage_eur", "age"]])

test[["potential", "value_eur", "wage_eur", "age"]] = standard_scaler.transform(test[["potential", "value_eur", "wage_eur", "age"]])

<ipython-input-44-083dccb1885a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[["potential", "value_eur", "wage_eur", "age"]] = standard_scaler.transform(df_train[["potential", "value_eur", "wage_eur", "age"]])


In [45]:
# partir df_train en df_train_train y df_train_valid
df_train_train, df_train_valid = train_test_split(df_train, test_size=0.3, random_state=0, shuffle=True, stratify=df_train.starter)

In [48]:
# Modelo prueba balanceado
model_lr_balanced = LogisticRegression(class_weight="balanced", random_state=0)
model_lr_balanced.fit(df_train_train[["potential", "value_eur", "wage_eur", "age"]],
          y=df_train_train.starter)
preds_lr_balanced = model_lr_balanced.predict(df_train_valid[["potential", "value_eur", "wage_eur", "age"]])

In [49]:
# medir metricas del nuevo modelo:
print_acc_rec_prec(df_train_valid.starter, preds_lr_balanced)

Accuracy: 0.6417751226797525
Recall: 0.6437724243977447
Precision: 0.5607142857142857


In [50]:
# Modelo prueba no balanceado
model_lr_balanced = LogisticRegression(class_weight=None, random_state=0)
model_lr_balanced.fit(df_train_train[["potential", "value_eur", "wage_eur", "age"]],
          y=df_train_train.starter)
preds_lr_balanced = model_lr_balanced.predict(df_train_valid[["potential", "value_eur", "wage_eur", "age"]])

# medir metricas del nuevo modelo:
print_acc_rec_prec(df_train_valid.starter, preds_lr_balanced)

Accuracy: 0.6360145082142095
Recall: 0.40389543823680163
Precision: 0.5920360631104433


In [51]:
# Modelo final
model_lr_balanced = LogisticRegression(class_weight="balanced", random_state=0)
model_lr_balanced.fit(df_train[["potential", "value_eur", "wage_eur", "age"]],
          y=df_train.starter)
preds_lr_balanced = model_lr_balanced.predict(test[["potential", "value_eur", "wage_eur", "age"]])

In [36]:
uploaded = files.upload()

Saving sample_submission.csv to sample_submission.csv


In [37]:
sample_submission = pd.read_csv("sample_submission.csv")

In [ ]:
test.head()

In [55]:
preds_lr_balanced

array([1., 1., 1., ..., 0., 0., 0.])

In [53]:
test['starter'] = preds_lr_balanced

In [54]:
test


,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,gender,starter
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,3.631766,18.641680,11.756556,1.964744,...,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png,male,1.0
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,3.473318,28.785195,9.821502,1.531306,...,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png,male,1.0
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,3.314870,10.575753,9.821502,2.398182,...,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png,male,1.0
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,3.314870,31.107204,9.821502,0.881150,...,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png,male,1.0
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,3.314870,30.251727,12.917589,1.097869,...,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png,male,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,261962,https://sofifa.com/player/261962/defu-song/220002,Song Defu,宋德福,CDM,47,-2.864597,-0.406129,-0.589090,-0.635883,...,46+2,48+2,15+2,https://cdn.sofifa.net/players/261/962/22_120.png,https://cdn.sofifa.net/teams/112541/60.png,https://cdn.sofifa.net/flags/cn.png,NaN,https://cdn.sofifa.net/flags/cn.png,male,0.0
19235,262040,https://sofifa.com/player/262040/caoimhin-port...,C. Porter,Caoimhin Porter,CM,47,-1.755462,-0.396352,-0.608441,-1.286039,...,44+2,48+2,14+2,https://cdn.sofifa.net/players/262/040/22_120.png,https://cdn.sofifa.net/teams/445/60.png,https://cdn.sofifa.net/flags/ie.png,NaN,https://cdn.sofifa.net/flags/ie.png,male,0.0
19236,262760,https://sofifa.com/player/262760/nathan-logue/...,N. Logue,Nathan Logue-Cunningham,CM,47,-2.389254,-0.398796,-0.608441,-0.852602,...,45+2,47+2,12+2,https://cdn.sofifa.net/players/262/760/22_120.png,https://cdn.sofifa.net/teams/111131/60.png,https://cdn.sofifa.net/flags/ie.png,NaN,https://cdn.sofifa.net/flags/ie.png,male,0.0
19237,262820,https://sofifa.com/player/262820/luke-rudden/2...,L. Rudden,Luke Rudden,ST,47,-1.597014,-0.396352,-0.608441,-1.286039,...,26+2,32+2,15+2,https://cdn.sofifa.net/players/262/820/22_120.png,https://cdn.sofifa.net/teams/111131/60.png,https://cdn.sofifa.net/flags/ie.png,NaN,https://cdn.sofifa.net/flags/ie.png,male,0.0


In [56]:
df_sub = test[['sofifa_id','starter']]

In [63]:
df_sub

,sofifa_id,starter
0,158023,1.0
1,188545,1.0
2,20801,1.0
3,190871,1.0
4,192985,1.0
...,...,...
19234,261962,0.0
19235,262040,0.0
19236,262760,0.0
19237,262820,0.0


In [ ]:
df_sub.drop(df_sub.columns[[0]],  axis = 'columns', index)


In [64]:
df_sub.to_csv('submission.csv', index = False)

In [ ]:
# medir metricas del nuevo modelo:
#print_acc_rec_prec(test.starter, preds_lr_balanced) # no se puede pq test no tiene variable starter